# 01. Create loop inventory
## Project: Bicycle node network loop analysis

This notebook creates a loop inventory from the input data set and calculates/plots basic descriptive statistics.

Contact: Michael Szell (michael.szell@gmail.com)

Created: 2024-01-24  
Last modified: 2024-02-14  

## To do

* Double-check cycle/edge lengths. For example 3-cycle east of Faxe
* X Incorporate gradients
* X Add cycle permutations for node-based analysis
* X Drop non-main nodes
* X Drop loops (they are really dangling links)
* X Find all simple cycles (bounded?-max length?) with networkX

## Parameters

In [ ]:
cycle_numnode_bound = 30  # From 30 it starts getting slow

PATH = {
    "data_in": "../data/input/faxe/network/",
    "data_out": "../data/processed/faxe/"
}

## Imports

In [ ]:
import geopandas as gpd
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from functools import reduce
import pickle 

## Functions

In [ ]:
def NormalizeData(data):
    return list((data - np.min(data)) / (np.max(data) - np.min(data)))

def getLayout(G, nodes_id, nodes_coords):
    named_vertex_list = G.vs()["name"]
    layout = []
    for n in named_vertex_list:
        pos = nodes_id.index(n)
        layout.append(nodes_coords[pos])
    return layout

def plotCheck(G, nodes_id, nodes_coords, vertex_size = 6, edge_color=None):
    fig, ax = plt.subplots()
    layout = getLayout(G, nodes_id, nodes_coords)
    if edge_color is None:
        ig.plot(G, target=ax, vertex_size=vertex_size, layout=layout);
    else:
        ig.plot(G, target=ax, vertex_size=vertex_size, layout=layout, edge_color=edge_color);
    return fig
    
def getCycleLength(c):
    l = 0
    cl = len(c)
    for i in range(cl):
        l += Gnx[c[i%cl]][c[(i+1)%cl]]["weight"]
    return l

def getCycleMaxSlope(c):
    ms = 0
    cl = len(c)
    for i in range(cl):
        ms = max([ms, Gnx[c[i%cl]][c[(i+1)%cl]]["max_slope"]])
    return ms

## Processing data

### Load data

In [ ]:
edges = gpd.read_file(PATH['data_in'] + 'edges_slope.gpkg')
nodes = gpd.read_file(PATH['data_in'] + 'nodes_edges_parallel.gpkg')
# Set CRS
edges.set_crs('epsg:25832')
nodes.set_crs('epsg:25832');

In [ ]:
edges.head()

In [ ]:
nodes.head()

In [ ]:
nodes_id = list(nodes.id)
nodes_x = list(nodes.geometry.x)
nodes_y = list(nodes.geometry.y)
nodes_coords = list(zip(NormalizeData(nodes_x), NormalizeData(nodes_y)))

In [ ]:
# Rename length to weight for igraph
edges = edges.rename(columns={"length": "weight"})
# Drop unused columns
used_columns = {"u":(), "v":(), "weight":(), "max_slope":()}
for c_name, _ in edges.items():
    if c_name not in used_columns:
        del edges[c_name]

# Reorder columns
edges = edges[['u','v','weight','max_slope']]

### Turn into igraph object

In [ ]:
G = ig.Graph.TupleList(edges.itertuples(index=False), directed=False, weights=False, edge_attrs = ['weight', 'max_slope'])

In [ ]:
G.summary()

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords)

### Drop self-loops

They are really dangling links which go outside the region, were mistakenly connected to themselves.

In [ ]:
G.simplify(multiple=True, loops=True, combine_edges=dict(weight=min, max_slope=max));

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords)

### Drop dangling nodes

In [ ]:
# Source: https://codereview.stackexchange.com/questions/284246/deletion-of-nodes-of-degree-1-from-a-python-igraph-graph
vertices = {v for v in G.vs.select(_degree_le=1)}
needs_to_be_checked = set(vertices)
while needs_to_be_checked:
    vertex = needs_to_be_checked.pop()
    for n_vertex in vertex.neighbors():
        if n_vertex in vertices \
                or sum(1 for v in n_vertex.neighbors() if v not in vertices) > 1:
            continue
        vertices.add(n_vertex)
        needs_to_be_checked.add(n_vertex)
G.delete_vertices(vertices)

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords)

### Drop degree 2 nodes

This should include all non-ismain nodes.

In [ ]:
nodes_nonismain = nodes.loc[nodes['ismain'] == 0]
nodes_nonismain = nodes_nonismain['node_id'].to_list()
# Turn to dict for fast finding
nodes_nonismain = {nodes_nonismain[i]: True for i in range(len(nodes_nonismain))} 

In [ ]:
to_delete_ids = []

# Unclear how to select nodes in igraph by name, so let's iterate through them
for v in G.vs:
#     if v["name"] in nodes_nonismain and v.degree() == 2:
    if v.degree() == 2:
        # Remember node to delete
        to_delete_ids.append(v.index)
        # Add a new edge that combines the deleted ones
        sumoflengths = v.incident()[0].attributes()["weight"] + v.incident()[1].attributes()["weight"]
        maxofslopes = max([v.incident()[0].attributes()["max_slope"], v.incident()[1].attributes()["max_slope"]])
        G.add_edge(v.neighbors()[0].index, v.neighbors()[1].index, weight=sumoflengths, max_slope=maxofslopes)

G.delete_vertices(to_delete_ids)

# Re-simplify
G.simplify(multiple=True, loops=True, combine_edges=dict(weight=min, max_slope=max));

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords)

### Get minimal cycle basis (=face cycles)

In [ ]:
# https://python.igraph.org/en/latest/api/igraph.GraphBase.html#minimum_cycle_basis
cyclebasis = {}
cid = 0
for c in G.minimum_cycle_basis():
    # Add some statistics
    ws = [G.es(eid)['weight'] for eid in c]
    cyclebasis[cid] = {"edges": c, "length": sum(reduce(lambda a, b: a + b, ws)), "numnodes": len(c)}
    cid += 1

In [ ]:
# Show longest cycle in minimal cycle basis (last has most nodes)
edge_colors = []
for e in G.es:
    if e.index in cyclebasis[max(cyclebasis.keys())]["edges"]:
        edge_colors.append("red")
    else:
        edge_colors.append("grey")

In [ ]:
fig = plotCheck(G, nodes_id, nodes_coords, vertex_size=6, edge_color=edge_colors)
plt.text(0,0.04, "Longest face loop highlighted: " + str(int(cyclebasis[max(cyclebasis.keys())]["length"]/1000)) + "km");
plt.tight_layout()

Getting all simple cycles has not yet been implemented in igraph, see:  
* https://github.com/igraph/igraph/issues/379  
* https://github.com/igraph/igraph/issues/1398  
Some potential progress here, but only for C, not Python:
* https://github.com/igraph/igraph/pull/2181

But they can be XORed through the cycle base.  

It has been implemented in networkX though: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.cycles.simple_cycles.html#networkx.algorithms.cycles.simple_cycles

Therefore, we do not use igraph's cycle basis, but go ahead with networkX.

### Get all cycles via nx

In [ ]:
Gnx = G.to_networkx()

In [ ]:
# # Sanity checks
# for (u, v, wt) in Gnx.edges.data('max_slope'):
#     print(f"({u}, {v}, {wt})")

In [ ]:
allcycles_unique = {}
nodes_done = set()
numcycles_unique = 0
allcycles_generator = nx.simple_cycles(Gnx, length_bound=cycle_numnode_bound)
for c in allcycles_generator:
    sourcenode = c[0]
    c_length = getCycleLength(c)
    c_max_slope = getCycleMaxSlope(c)
    numcycles_unique += 1
    if sourcenode in nodes_done:
        allcycles_unique[sourcenode]["cycles"].append(c)
        allcycles_unique[sourcenode]["lengths"].append(c_length)
        allcycles_unique[sourcenode]["numnodes"].append(len(c))
        allcycles_unique[sourcenode]["max_slopes"].append(c_max_slope)
    else:
        allcycles_unique[sourcenode] = {"cycles": [c], "lengths": [c_length], "numnodes": [len(c)], "max_slopes": [c_max_slope]}
        nodes_done.add(sourcenode)
print("Found " + str(numcycles_unique) + " unique cycles for length bound " + str(cycle_numnode_bound))

In [ ]:
allcycles = {}
nodes_done = set()
numcycles = 0
allcycles_generator = nx.simple_cycles(Gnx, length_bound=cycle_numnode_bound)
for c in allcycles_generator:
    sourcenode = c[0]
    c_length = getCycleLength(c)
    c_max_slope = getCycleMaxSlope(c)
    for sourcenode in c:
        numcycles += 1
        if sourcenode in nodes_done:
            allcycles[sourcenode]["cycles"].append(c)
            allcycles[sourcenode]["lengths"].append(c_length)
            allcycles[sourcenode]["numnodes"].append(len(c))
            allcycles[sourcenode]["max_slopes"].append(c_max_slope)
        else:
            allcycles[sourcenode] = {"cycles": [c], "lengths": [c_length], "numnodes": [len(c)], "max_slopes": [c_max_slope]}
            nodes_done.add(sourcenode)
print("Found " + str(numcycles) + " cycles for length bound " + str(cycle_numnode_bound))

In [ ]:
allcyclelengths = np.zeros(numcycles)
allcyclenumnodes = np.zeros(numcycles, dtype=int)
allcyclemaxslopes = np.zeros(numcycles)
i = 0
for j in allcycles:
    l = len(allcycles[j]["lengths"])
    allcyclelengths[i:i+l] = allcycles[j]["lengths"]
    allcyclenumnodes[i:i+l] = allcycles[j]["numnodes"]
    allcyclemaxslopes[i:i+l] = allcycles[j]["max_slopes"]
    i += l

In [ ]:
fig = plt.figure(figsize=(8, 3))
axes1 = fig.add_axes([0.1, 0.1, 0.35, 0.8])
axes2 = fig.add_axes([0.55, 0.1, 0.35, 0.8])

axes1.hist(allcyclelengths, density=True)
axes1.set_xlabel('Length [m]')
axes1.set_ylabel('Probability')
axes1.set_title('Cycle lengths')

axes2.hist(allcyclenumnodes, density=True, bins=list(range(cycle_numnode_bound+1)))
axes2.set_xlabel('Nodes')
axes2.set_title('Nodes per cycle')
axes2.set_xlim([0, cycle_numnode_bound+0.5])

plt.text(cycle_numnode_bound/20,0.01, "Bound: " + str(cycle_numnode_bound))
plt.text(cycle_numnode_bound/20,0.04, "Cycles: " + str(numcycles));

In [ ]:
fig = plt.figure(figsize=(8, 3))
axes1 = fig.add_axes([0.1, 0.1, 0.35, 0.8])
axes2 = fig.add_axes([0.55, 0.1, 0.35, 0.8])

histxy = axes1.hist([c["length"] for c in cyclebasis.values()], density=False)
axes1.set_xlabel('Length [m]')
axes1.set_ylabel('Frequency')
axes1.set_title('Face cycle lengths')
axes1.plot([15000,15000], [0,max(histxy[0])], ":k")
axes1.plot([30000,30000], [0,max(histxy[0])], ":k")
massinallowedrange = round(sum(histxy[0][-6:])/sum(histxy[0][:])*100) # Should be 90%
axes1.text(22500, max(histxy[0]), str(massinallowedrange) + "%", horizontalalignment='center', verticalalignment='top')

axes2.hist([c["numnodes"] for c in cyclebasis.values()], density=False, bins=list(range(cycle_numnode_bound+1)))
axes2.set_xlabel('Nodes')
axes2.set_title('Nodes per face cycle');

## Save cycle inventory

In [ ]:
with open(PATH['data_out'] + 'cycleinventory_'+str(cycle_numnode_bound)+'.pkl', 'wb') as f:
    pickle.dump(allcycles, f)
    pickle.dump(allcyclelengths, f)
    pickle.dump(allcyclenumnodes, f)
    pickle.dump(allcyclemaxslopes, f)
    pickle.dump(G, f)
    pickle.dump(cycle_numnode_bound, f)
    pickle.dump(nodes_id, f)
    pickle.dump(nodes_coords, f)
    pickle.dump(numcycles, f)